In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
raw_data_path=os.path.join('data','raw')
df=pd.read_csv(os.path.join(raw_data_path,'fer2013.csv'))

In [9]:
X=[]
for i in range(len(df)):
    X.append(df.loc[i,'pixels'].split(' '))
X=np.array(X,np.float32)/255.
y=df.emotion.values

In [10]:
import tensorflow as tf

In [11]:
X.shape

(35887, 2304)

In [15]:
X_train=X[np.where((df.Usage=='Training') & ((df.emotion==0) | (df.emotion==1)))]
y_train=y[np.where((df.Usage=='Training')& ((df.emotion==0) | (df.emotion==1)))]
X_test=X[np.where((df.Usage!='Training')& ((df.emotion==0) | (df.emotion==1)))]
y_test=y[np.where((df.Usage!='Training')& ((df.emotion==0) | (df.emotion==1)))]

In [16]:
X_test.shape

(1069, 2304)

In [17]:
X_train.shape

(4431, 2304)

In [33]:
def balance_classes(X,y):
    X1=X[np.where(y==0)]
    X2=X[np.where(y==1)]
    X2=np.repeat(X2,9,axis=0)
    X=np.concatenate((X1,X2),axis=0)
    y1=y[np.where(y==0)]
    y2=y[np.where(y==1)]
    y2=np.repeat(y2,9,axis=0)
    y=np.concatenate((y1,y2),axis=0)
    i=[i for i in range(len(y))]
    np.random.shuffle(i)
    return X[i],y[i]

In [36]:
X_train,y_train=balance_classes(X_train,y_train)

In [39]:
np.random.randn(2,2)

array([[-0.36431416,  0.57886118],
       [-0.4155075 ,  0.46718151]])

In [66]:
class ANN(object):
    def __init__(self,lr,epoch,hu):
        self.lr=lr
        self.epoch=epoch
        self.hu=hu
    def relu(self,z):
        return z*(z>0)
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    def forward(self,X,W1,b1,W2,b2):
        z=self.relu(X.dot(W1)+b1)
        return z,self.sigmoid(z.dot(W2)+b2)
    def cost(self,y,y_p):
        return -(y*np.log(y_p)+(1-y)*np.log(1-y_p)).sum()
    def fit(self,X,y,fig=False):
        N,D=X.shape
        W1=np.random.randn(D,self.hu)
        b1=np.random.randn(self.hu)
        W2=np.random.randn(self.hu)
        b2=np.random.randn()
        tr_i=np.random.choice(N,int(N*0.9),replace=False)
        t_i=[i for i in range(N) if i not in tr_i]
        X_t=X[tr_i]
        y_t=y[tr_i]
        X_v=X[t_i]
        y_v=y[t_i]
        c_t=[]
        c_v=[]
        cl_t=[]
        cl_v=[]
        for i in range(self.epoch):
            z,y_p_t=self.forward(X_t,W1,b1,W2,b2)
            _,y_p_v=self.forward(X_v,W1,b1,W2,b2)
            dy=y_p_t-y_t
            W2=W2-self.lr*(z.T.dot(dy))
            b2=b2-self.lr*(dy).sum(axis=0)
            dz=np.outer(dy,W2)*(z>0)
            W1=W1-self.lr*X_t.T.dot(dz)
            b1=b1-self.lr*(dz).sum(axis=0)
            c_t.append(self.cost(y_t,y_p_t))
            c_v.append(self.cost(y_v,y_p_v))
            cl_t.append(np.mean(y_t==np.round(y_p_t)))
            cl_v.append(np.mean(y_v==np.round(y_p_v)))
            if i % 100==0:
                print('Epoch {0} Train Cost , Classification : {1}  , {2} Test Cost , Classification : {3}  , {4}'.format(i,
                                                                                                                          c_t[i],
                                                                                                                         cl_t[i],
                                                                                                                         c_v[i],
                                                                                                                         cl_v[i]))
        self.W2=W2
        self.W1=W1
        self.b2=b2
        self.b1=b1
        if fig==True:
            plt.plot(c_t,label='Train Cost')
            plt.plot(c_v,label='Test Cost')
            plt.show()
            plt.plot(cl_t,label='Train Class')
            plt.plot(cl_v,label='Test Class')
            plt.show()
    def predict(self,X):
        _,y_p=forward(X,self.W1,self.b1,self.W2,self.b2)
        return np.round(y_p)
    def score(self,y,y_p):
        return np.mean(y==y_p)


In [67]:
ann=ANN(0.0000001,10000,5)

In [68]:
ann.fit(X_train,y_train,fig=True)

Epoch 0 Train Cost , Classification : 129335.11795332075  , 0.5075066647958468 Test Cost , Classification : 15474.323145281198  , 0.4659090909090909
Epoch 100 Train Cost , Classification : 5174.320714924064  , 0.5347270941490108 Test Cost , Classification : 572.5982321581505  , 0.5454545454545454
Epoch 200 Train Cost , Classification : 5058.814987576776  , 0.5463729479444367 Test Cost , Classification : 564.6098397112146  , 0.5265151515151515
Epoch 300 Train Cost , Classification : 5022.02548591529  , 0.5414620457415462 Test Cost , Classification : 560.8260730609929  , 0.523989898989899
Epoch 400 Train Cost , Classification : 4999.095790429969  , 0.5427248491651466 Test Cost , Classification : 558.3084392419125  , 0.5265151515151515
Epoch 500 Train Cost , Classification : 4981.6475586089  , 0.545811701978392 Test Cost , Classification : 556.264113395386  , 0.5353535353535354
Epoch 600 Train Cost , Classification : 4968.850798017544  , 0.5434264066227024 Test Cost , Classification : 554

C:\Users\ytnvj\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  
C:\Users\ytnvj\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in multiply
  


Epoch 4600 Train Cost , Classification : nan  , 0.49235302371264206 Test Cost , Classification : nan  , 0.5328282828282829
Epoch 4700 Train Cost , Classification : nan  , 0.49235302371264206 Test Cost , Classification : nan  , 0.5328282828282829
Epoch 4800 Train Cost , Classification : nan  , 0.49235302371264206 Test Cost , Classification : nan  , 0.5328282828282829
Epoch 4900 Train Cost , Classification : nan  , 0.49235302371264206 Test Cost , Classification : nan  , 0.5328282828282829
Epoch 5000 Train Cost , Classification : 1322456.719478807  , 0.5086291567279361 Test Cost , Classification : 158378.70827452024  , 0.4671717171717172
Epoch 5100 Train Cost , Classification : nan  , 0.49235302371264206 Test Cost , Classification : nan  , 0.5328282828282829
Epoch 5200 Train Cost , Classification : nan  , 0.491370843272064 Test Cost , Classification : nan  , 0.5328282828282829
Epoch 5300 Train Cost , Classification : nan  , 0.491370843272064 Test Cost , Classification : nan  , 0.532828282

KeyboardInterrupt: 

In [69]:
df.emotion.value_counts()

3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64